# Parcial 1

¡El mejor celular para mamá!
Mamá cumple años en unos días y aún no le he comprado el regalo. Ella quiere cambiar
su celular desde hace un tiempo por lo que quizás ese sea el mejor regalo que le podría
dar. Al ingresar a una plataforma de comercio electrónico he podido descargar la lista de
más de 900 celulares publicados. Quisiera aplicar los conocimientos que he adquirido en
mi curso de ciencia de datos para escoger la mejor opción para ella.

Mamá no es muy tecnológica por lo que no hay necesidad de regalarle un celular de gama
alta. Probablemente un celular de máximo $1.200.000 sea más que suficiente, aunque si
lo puedo pagar a cuotas mucho mejor. Además de que las características del celular
como la marca, el color, el tamaño de la pantalla, entre otras, sean las ideales para ella,
siempre me gusta revisar que el vendedor tenga buena reputación (ojalá que sea una
tienda oficial), que ofrezca garantía de al menos 6 meses y que el envío sea gratis. Su
cumpleaños es muy pronto así que lo mejor es que el vendedor esté en los alrededores
de Bogotá para que el envío sea rápido. Para propósitos del ejercicio, lo anterior no debe
considerarse como restricciones estrictas sino más bien como características o
condiciones de compra deseadas.

Dentro de lo que logré descargar cuento con los siguientes conjuntos de datos: (1) la lista
de más de 900 celulares publicados a la fecha en la plataforma, (2) todo el detalle de cada
uno de los celulares de esta lista, y (3) la información básica de los diferentes vendedores

## 1

### a. Reporte de calidad

### b. Diagrama de bloques

## 2

### a. Análisis exploratorio de las variables relevantes

#### I. Análisis univariado

#### II. Análisis bivariado

### b. ¿Qué tantas opciones (variabilidad) en función de la marca, precio u otros dos atributos relevantes hay en los celulares disponibles?

## 3

#### a. Diseño de la estrategia

#### *¿ Cuales considera que son los pasos más importantes para seleccionar el celular ideal?*

#### b. Proceso de selección con prueba estadística con significancia

#### c. Proceso para recomendar el celular que se debe comprar.